<a href="https://colab.research.google.com/github/saksham751/Neural-Network-from-Scratch/blob/Comments/neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets.mnist import load_data
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Neural Network Class

In [2]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    def forward_propagation(self, input):
        raise NotImplementedError
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [3]:
class completeLayer(Layer):
  
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size)
        self.bias = np.random.rand(1, output_size)

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        #print(output_error.shape, self.weights.T.shape)
        #print(self.input.T.shape, output_error.shape)
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T.reshape(self.input.T.shape[0],1), output_error)
        
        self.weights -= learning_rate * weights_error
        self.bias = self.bias-learning_rate * output_error
        return input_error

In [4]:

class activationFunction(Layer):
    def __init__(self, activation, activationDer):
        self.activation = activation
        self.activationDer = activationDer

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activationDer(self.input) * output_error

In [5]:
    def sig(x):
      return 1/(1 + np.exp(-x))

    def sig_der(x):
      return sig(x)*(1-sig(x))
  
    def tanh(x):
      return np.tanh(x)

    def tanh_prime(x):
      return 1-np.tanh(x)**2

    def relu(x):
      return np.maximum(x, 0)

    def relu_der(x):
      return 1. * (x > 0)

    def linear(x):
      return x

    def linear_der(x):
      return 1

    

In [6]:

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mseDer(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [7]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):

        samples = len(input_data)
        result = []
        for i in range(samples):

            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    def fit(self, x_train, y_train, epochs, learning_rate):

        samples = len(x_train)
        for i in range(epochs):
            err = 0
            for j in range(samples):
  
                output = x_train[j]
                
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                err += self.loss(y_train[j], output)

                error = self.loss_prime(y_train[j], output)
                #error.reshape(784,1)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
                #print('error=%f'%(err/j))

            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [8]:
def classAccuracy( y_test,res):
  cor=0
  for x,y in zip(y_test,res):
    cor+=x==y
  acc=cor/len(res)*100
  return acc

In [9]:
def mseRegression(y_test,res):
  errors = mean_squared_error(y_test, res)
  return errors

# Classification Task

In [10]:
#Loading the iris dataset
iris = datasets.load_iris()
x, y = iris.data, iris.target

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123)

In [12]:
#calculating the number of classes in the dataset
x_train.shape[1]
classes=np.unique(y_train).size
y_train

array([2, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 0, 2, 2, 2, 2, 2, 1, 0, 0, 2, 0,
       0, 1, 1, 1, 1, 2, 1, 2, 0, 2, 1, 0, 0, 2, 1, 2, 2, 0, 1, 1, 2, 0,
       2, 1, 1, 0, 2, 2, 0, 0, 1, 1, 2, 0, 0, 1, 0, 1, 2, 0, 2, 0, 0, 1,
       0, 0, 1, 2, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 0,
       0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 1, 2, 1, 2, 2, 0, 1, 0, 2, 2,
       1, 1, 2, 2, 1, 0, 1, 1, 2, 2])

In [13]:
#converting to binary encoded categorical values
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_train.shape

(120, 3)

In [14]:
network1 = Network()
x1=x_train.shape[1]
x2=4
network1.add(completeLayer(x1, x2))
network1.add(activationFunction(tanh, tanh_prime))
network1.add(completeLayer(x2, classes))
network1.add(activationFunction(tanh, tanh_prime))


In [15]:
x_train.shape

(120, 4)

In [16]:
network1.use(mse, mseDer)

network1.fit(x_train ,y_train, epochs=1000, learning_rate=0.001)

epoch 1/1000   error=0.652963
epoch 2/1000   error=0.652724
epoch 3/1000   error=0.652476
epoch 4/1000   error=0.652218
epoch 5/1000   error=0.651949
epoch 6/1000   error=0.651669
epoch 7/1000   error=0.651376
epoch 8/1000   error=0.651071
epoch 9/1000   error=0.650751
epoch 10/1000   error=0.650417
epoch 11/1000   error=0.650066
epoch 12/1000   error=0.649698
epoch 13/1000   error=0.649312
epoch 14/1000   error=0.648906
epoch 15/1000   error=0.648478
epoch 16/1000   error=0.648027
epoch 17/1000   error=0.647550
epoch 18/1000   error=0.647046
epoch 19/1000   error=0.646512
epoch 20/1000   error=0.645944
epoch 21/1000   error=0.645341
epoch 22/1000   error=0.644699
epoch 23/1000   error=0.644013
epoch 24/1000   error=0.643279
epoch 25/1000   error=0.642493
epoch 26/1000   error=0.641647
epoch 27/1000   error=0.640736
epoch 28/1000   error=0.639753
epoch 29/1000   error=0.638687
epoch 30/1000   error=0.637530
epoch 31/1000   error=0.636270
epoch 32/1000   error=0.634892
epoch 33/1000   e

In [17]:
x_test

array([[6.3, 2.5, 4.9, 1.5],
       [6.8, 3. , 5.5, 2.1],
       [6.4, 2.8, 5.6, 2.2],
       [5.6, 3. , 4.1, 1.3],
       [4.9, 3.6, 1.4, 0.1],
       [6. , 3. , 4.8, 1.8],
       [6.3, 2.3, 4.4, 1.3],
       [4.4, 3.2, 1.3, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [5.5, 2.6, 4.4, 1.2],
       [6.9, 3.1, 5.1, 2.3],
       [5.5, 4.2, 1.4, 0.2],
       [5.2, 2.7, 3.9, 1.4],
       [6.5, 3. , 5.5, 1.8],
       [7.7, 3. , 6.1, 2.3],
       [6.5, 3. , 5.8, 2.2],
       [5.5, 3.5, 1.3, 0.2],
       [4.3, 3. , 1.1, 0.1],
       [6.1, 2.9, 4.7, 1.4],
       [4.8, 3. , 1.4, 0.3],
       [5.2, 3.4, 1.4, 0.2],
       [6.3, 2.8, 5.1, 1.5],
       [4.8, 3.4, 1.9, 0.2],
       [6.1, 3. , 4.9, 1.8],
       [5.1, 3.8, 1.6, 0.2],
       [5.4, 3.4, 1.7, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.6, 2.8, 4.9, 2. ],
       [7.7, 3.8, 6.7, 2.2],
       [5. , 3.6, 1.4, 0.2]])

In [18]:
out = network1.predict(x_test)
out=[x[0] for x in out]

In [19]:
out

[array([0.30422005, 0.37063315, 0.32545913]),
 array([0.30421616, 0.37062946, 0.32546122]),
 array([0.30421718, 0.37063042, 0.32546067]),
 array([0.30422011, 0.37063249, 0.32545916]),
 array([0.30420391, 0.3706238 , 0.32546697]),
 array([0.3042179 , 0.37063091, 0.3254603 ]),
 array([0.30422282, 0.37063582, 0.32545764]),
 array([0.30419566, 0.3706187 , 0.32547077]),
 array([0.30420791, 0.37062793, 0.32546438]),
 array([0.3042249 , 0.37063705, 0.32545659]),
 array([0.30421592, 0.37062923, 0.32546134]),
 array([0.3042118 , 0.37062649, 0.32546338]),
 array([0.30422642, 0.3706376 , 0.32545586]),
 array([0.30421664, 0.37062989, 0.32546096]),
 array([0.3042154 , 0.37062876, 0.32546162]),
 array([0.3042164 , 0.37062968, 0.32546109]),
 array([0.30421498, 0.37062886, 0.32546171]),
 array([0.30418257, 0.37061697, 0.32547633]),
 array([0.30421846, 0.37063146, 0.32546   ]),
 array([0.30421932, 0.37063195, 0.32545929]),
 array([0.3042139 , 0.37062846, 0.32546218]),
 array([0.30421809, 0.37063123, 0.

In [20]:
res=np.argmax(out,axis=1)
res

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])

In [21]:
y_test

array([1, 2, 2, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 2, 2, 2, 0, 0, 1, 0, 0, 2,
       0, 2, 0, 0, 0, 2, 2, 0])

In [22]:
classAccuracy(y_test,res)

20.0

# Regression Task

In [23]:
#loading the diabetes dataset
db = datasets.load_diabetes()
x, y = db.data, db.target

In [24]:
#SPlitting data for training and testing
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123)

In [25]:
x_train
y_train

array([170.,  77.,  49.,  66., 144., 113., 233., 162., 121.,  88., 302.,
       128.,  52., 178.,  90., 164., 135., 103., 200., 178., 261.,  64.,
        59.,  79.,  47., 107.,  39., 151., 104., 217., 232.,  55., 245.,
       131.,  64., 222., 249., 128., 293., 138., 273., 158., 168., 103.,
       140., 146., 277., 116., 233.,  94., 295., 191., 241., 154., 141.,
       252., 198., 288., 101., 179., 129., 174.,  91.,  84., 177., 219.,
        67., 113.,  70.,  53., 150.,  40., 258., 168., 283., 252.,  43.,
        74.,  78., 209., 270.,  90.,  73., 310., 102., 115.,  85., 104.,
        68., 101., 139.,  63.,  72.,  58., 131.,  71.,  98.,  88., 277.,
       104., 167., 108., 111., 214., 135., 196., 189.,  77., 225.,  60.,
        67., 292., 143., 281.,  25., 200.,  91., 263.,  95., 144., 232.,
       109.,  51.,  64., 283., 259., 214.,  60., 111., 258.,  68., 174.,
       118., 113., 306.,  70.,  78., 191., 220., 183., 124.,  44.,  63.,
       114., 229.,  77., 168., 262.,  91.,  85., 12

In [26]:
network2 = Network()
x1=x_train.shape[1]
x2=x1*2
network2.add(completeLayer(x1, x2))
network2.add(activationFunction(relu, relu_der))
network2.add(completeLayer(x2, 1))
network2.add(activationFunction(linear, linear_der))


In [27]:
network2.use(mse, mseDer)

network2.fit(x_train ,y_train, epochs=1000, learning_rate=0.0002)

epoch 1/1000   error=6288.092856
epoch 2/1000   error=4680.471684
epoch 3/1000   error=4156.648249
epoch 4/1000   error=3865.221896
epoch 5/1000   error=3704.792430
epoch 6/1000   error=3600.910660
epoch 7/1000   error=3527.684870
epoch 8/1000   error=3474.766989
epoch 9/1000   error=3435.139339
epoch 10/1000   error=3404.276623
epoch 11/1000   error=3380.476372
epoch 12/1000   error=3361.329648
epoch 13/1000   error=3344.738679
epoch 14/1000   error=3331.984328
epoch 15/1000   error=3320.471327
epoch 16/1000   error=3310.267094
epoch 17/1000   error=3301.312693
epoch 18/1000   error=3295.065335
epoch 19/1000   error=3285.324638
epoch 20/1000   error=3279.560262
epoch 21/1000   error=3275.041701
epoch 22/1000   error=3266.535709
epoch 23/1000   error=3263.488484
epoch 24/1000   error=3258.274654
epoch 25/1000   error=3254.810840
epoch 26/1000   error=3251.262191
epoch 27/1000   error=3247.354403
epoch 28/1000   error=3243.039626
epoch 29/1000   error=3238.561768
epoch 30/1000   error=3

In [32]:
out = network2.predict(x_test)
out=[x[0] for x in out]
print(out)

[array([130.85741767]), array([121.50257848]), array([201.9500629]), array([141.20006931]), array([138.56848398]), array([131.0836018]), array([249.52887114]), array([97.68664049]), array([85.45192508]), array([90.38347865]), array([109.42535518]), array([159.52373001]), array([141.2247893]), array([175.31612252]), array([204.9627637]), array([143.6510996]), array([107.0806162]), array([136.70560756]), array([147.82149685]), array([175.37785587]), array([144.31028049]), array([223.97886403]), array([124.31181648]), array([75.45989093]), array([97.8072905]), array([157.13390385]), array([86.11886048]), array([101.59733229]), array([126.38769105]), array([148.5298218]), array([104.65930376]), array([212.58407871]), array([178.09233382]), array([201.39547601]), array([162.49862836]), array([86.33862838]), array([75.45989093]), array([100.81908762]), array([208.6648675]), array([90.02709389]), array([175.26708578]), array([83.35787586]), array([100.93793846]), array([119.66110512]), array(

In [30]:
mseRegression(y_test,out)

3021.0919581774865